In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('new1_cyber_train.csv')
test_df = pd.read_csv('new1_cyber_test.csv')

In [2]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from transformers import AlbertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Apply stemming
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

train_df['text'] = train_df['text'].apply(preprocess_text)
test_df['text'] = test_df['text'].apply(preprocess_text)

# Tokenization for ALBERT
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)

# Encoding as numbers for training dataset
encoded_train_texts = tokenizer(train_df['text'].tolist(), padding=True, truncation=True, return_tensors='tf')
X_train_padded = pad_sequences(encoded_train_texts['input_ids'], padding='post')
y_train = train_df['label'].to_numpy()

# Encoding as numbers for testing dataset
encoded_test_texts = tokenizer(test_df['text'].tolist(), padding=True, truncation=True, return_tensors='tf')
X_test_padded = pad_sequences(encoded_test_texts['input_ids'], padding='post')
y_test = test_df['label'].to_numpy()

# Building the ALBERT model
model = TFBertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=2)

# Compile the model (define optimizer, loss, and metric)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Training the model
history = model.fit(
  {'input_ids': X_train_padded, 'attention_mask': encoded_train_texts['attention_mask']},
  y_train,
  epochs=3,
  batch_size=32,
  validation_split=0.1
)

# Evaluating the model on the testing dataset
eval_result = model.evaluate(
  {'input_ids': X_test_padded, 'attention_mask': encoded_test_texts['attention_mask']},
  y_test
)
print(f"Test accuracy: {eval_result[1]}")


spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nitin\.cache\huggingface\hub\models--albert-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using a model of type albert to instantiate a model of type bert. This is not supported 

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.bias', 'predictions.LayerNorm.bias', 'albert.encoder.embedding_hidden_mapping_in.weight', 'predictions.LayerNorm.weight', 'predictions.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.weight', 'albert.embeddings.LayerNorm.bias', 'albert.encoder.embedding_hidden_mapping_in.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.dense.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.ffn.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.dense.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias', 'predictions.decoder.bias', 'albert.embeddings.word_embeddings.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight', 'albert.encoder.albert_layer_groups.0.albert_lay

Epoch 1/3

26/26 [==============================] - 691s 25s/step - loss: 0.7093 - accuracy: 0.5166 - val_loss: 0.6712 - val_accuracy: 0.6264
Epoch 2/3
26/26 [==============================] - 651s 25s/step - loss: 0.6439 - accuracy: 0.6782 - val_loss: 0.5902 - val_accuracy: 0.7363
Epoch 3/3
8/8 [==============================] - 44s 5s/step - loss: 0.7901 - accuracy: 0.5664
Test accuracy: 0.5663716793060303
